In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
# 文件路径列表

file_paths = [
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_3.csv',
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_7.csv',
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_15.csv',
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_30.csv'
]

# 标签列名
label_columns = ['未来7日涨幅']

# 存储每个标签的准确率
accuracies = {label: [] for label in label_columns}

In [3]:
# 创建包含注意力机制的模型
def create_advanced_model_with_attention(input_shape):
    inputs = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(inputs)
    x = Dropout(0.3)(x)
    x = LSTM(32, return_sequences=True)(x)
    x = Dropout(0.3)(x)
    
    # Attention mechanism
    attention = Attention()([x, x])
    x = Concatenate(axis=-1)([x, attention])
    
    x = LSTM(32, return_sequences=False)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
# 遍历每个数据集
for file_path in file_paths:
    print(f"Processing file: {file_path}")
    
    # 读取数据
    data = pd.read_csv(file_path)
    data = data.dropna()

    # 特征和标签
    features = data.iloc[:, 1:192]  # n1-n47, p1-p146
    labels = {label: (data[label] > 0).astype(int) for label in label_columns}

    # 标准化
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # Reshape data for LSTM input
    features_scaled_reshaped = features_scaled.reshape((features_scaled.shape[0], features_scaled.shape[1], 1))

    # 定义回调函数
    early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

    # 对每个标签进行训练和评估
    for label_name, label_data in labels.items():
        print(f"Training for label: {label_name}")
        
        X_train, X_test, y_train, y_test = train_test_split(features_scaled_reshaped, label_data, test_size=0.2, random_state=42)

        # 创建改进后的模型
        model = create_advanced_model_with_attention((features_scaled.shape[1], 1))

        # 训练模型
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr], verbose=1)

        # 预测并评估
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies[label_name].append(accuracy)

Processing file: C:\Users\mjy\Desktop\Project_MSCM\INPUT_3.csv
Training for label: 未来7日涨幅


Epoch 1/100


17/17 [==============================] - 10s 211ms/step - loss: 0.6910 - accuracy: 0.5212 - val_loss: 0.6842 - val_accuracy: 0.5588 - lr: 0.0010
Epoch 2/100
17/17 [==============================] - 2s 100ms/step - loss: 0.6879 - accuracy: 0.5470 - val_loss: 0.6815 - val_accuracy: 0.5588 - lr: 0.0010
Epoch 3/100
17/17 [==============================] - 2s 99ms/step - loss: 0.6894 - accuracy: 0.5506 - val_loss: 0.6856 - val_accuracy: 0.5588 - lr: 0.0010
Epoch 4/100
17/17 [==============================] - 2s 104ms/step - loss: 0.6878 - accuracy: 0.5506 - val_loss: 0.6833 - val_accuracy: 0.5588 - lr: 0.0010
Epoch 5/100
17/17 [==============================] - 2s 107ms/step - loss: 0.6852 - accuracy: 0.5506 - val_loss: 0.6821 - val_accuracy: 0.5588 - lr: 0.0010
Epoch 6/100
17/17 [==============================] - 2s 104ms/step - loss: 0.6883 - accuracy: 0.5488 - val_loss: 0.6839 - val_

In [5]:
# 输出所有文件的准确率
for label_name, accuracy_list in accuracies.items():
    for i, accuracy in enumerate(accuracy_list):
        print(f"File: {file_paths[i]}, Label: {label_name}, Accuracy: {accuracy:.4f}")

File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_3.csv, Label: 未来7日涨幅, Accuracy: 0.5529
File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_7.csv, Label: 未来7日涨幅, Accuracy: 0.5325
File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_15.csv, Label: 未来7日涨幅, Accuracy: 0.5893
File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_30.csv, Label: 未来7日涨幅, Accuracy: 0.6606
